This notebook will go through the pipeline for normalizing an fMRI dataset into the MNI template space.

In [ ]:
import os,shutil
import sklearn.metrics

try:
    datadir=os.environ['FMRIDATADIR']
    assert not datadir==''
except:
    datadir='/Users/poldrack/data_unsynced/myconnectome/sub00001'

print 'Using data from',datadir

%matplotlib inline

from nipype.interfaces import fsl
import nibabel
import numpy
import nilearn.plotting
import matplotlib.pyplot as plt
from nipype.interfaces import ants


In [ ]:
subject='ses014'  
# note - we have to use the anatomy from a different session'
anat_subject='ses018'
bolddir=os.path.join(datadir,'ds031/sub00001',subject,
        'functional')

meanbold_unwarped_file=os.path.join(bolddir,
    'sub00001_ses014_task002_run001_bold_mcf_brain_unwarped_mean.nii.gz')
bold_file=os.path.join(bolddir,
    'sub00001_ses014_task002_run001_bold_mcf_brain_unwarped.nii.gz')
anatomydir=os.path.join(datadir,'ds031/sub00001',anat_subject,
        'anatomy')
t1file=os.path.join(anatomydir,'sub00001_ses018_T1w_001.nii.gz')


Perform brain extraction using BET

In [ ]:
t1brainfile=t1file.replace('.nii.gz','_brain.nii.gz')
btr = fsl.BET(frac=0.3)
btr.inputs.in_file = t1file
btr.inputs.out_file=t1brainfile
btr.inputs.mask=True
if not os.path.exists(btr.inputs.out_file):
    res = btr.run() 

Visualize brain mask overlaid on anatomy

In [ ]:
brainmask=nibabel.load(t1brainfile.replace('.nii.gz','_mask.nii.gz'))
maskdata=brainmask.get_data()
t1img=nibabel.load(t1file)

mask_display=nilearn.plotting.plot_epi(t1img,cmap='gray')
mask_display.add_contours(brainmask, levels=[.5]) 

Perform bias field correction on T1-weighted image using FSL's FAST.

In [ ]:
bfcfile=os.path.join(anatomydir,'sub00001_ses018_T1w_001_brain_FAST_restore.nii.gz')

if not os.path.exists(bfcfile):
    fastr = fsl.FAST(verbose=True)
    fastr.inputs.in_files = t1brainfile
    fastr.inputs.output_biascorrected=True
    fastr.inputs.out_basename=t1brainfile.replace('.nii.gz','_FAST')
    try:
        out = fastr.run() 
    except:
        pass # FAST throws an error even if it finishes OK
else:
    print 'using existing bias-field corrected data'
# rename wmseg so that BFC will find it
wm_maskfile=os.path.join(anatomydir,'sub00001_ses018_T1w_001_brain_FAST_restore_wmseg.nii.gz')
if not os.path.exists(wm_maskfile):
    shutil.copy(os.path.join(anatomydir,'sub00001_ses018_T1w_001_brain_FAST_pve_2.nii.gz'),
            wm_maskfile)

Visualize white matter mask

In [ ]:
wm_mask=nibabel.load(wm_maskfile)

mask_display=nilearn.plotting.plot_epi(t1img,cmap='gray')
mask_display.add_contours(wm_mask, levels=[.5]) 

Register mean unwarped functional to anatomy using corratio

In [ ]:
if not os.path.exists(meanbold_unwarped_file.replace('.nii.gz','_reg2t1.mat')):

    flt = fsl.FLIRT(cost_func='corratio',dof=7)
    flt.inputs.in_file = meanbold_unwarped_file
    flt.inputs.reference = bfcfile
    flt.inputs.out_file = meanbold_unwarped_file.replace('.nii.gz','_reg2t1.nii.gz')
    flt.inputs.out_matrix_file = meanbold_unwarped_file.replace('.nii.gz','_reg2t1.mat')
    flt.inputs.wm_seg=wm_maskfile
    flt.run()
else:
    print 'using existing corratio registration'

Register mean unwarped functional to anatomy using BBR (using the matrix from the corratio registration as a starting point)

In [ ]:
bbrmat=meanbold_unwarped_file.replace('.nii.gz','_bbreg2t1.mat')
if not os.path.exists(bbrmat):
    
    flt = fsl.FLIRT(cost_func='bbr',verbose=6,dof=7)
    flt.inputs.in_file = meanbold_unwarped_file
    flt.inputs.reference = bfcfile
    flt.inputs.schedule = os.path.join(os.getenv('FSLDIR'),
                                                'etc/flirtsch/bbr.sch')

    flt.inputs.out_file = meanbold_unwarped_file.replace('.nii.gz','_bbreg2t1.nii.gz')
    flt.inputs.in_matrix_file = meanbold_unwarped_file.replace('.nii.gz','_reg2t1.mat')
    flt.inputs.out_matrix_file = bbrmat
    flt.inputs.wm_seg=wm_maskfile
    # NOTE: This takes a VERY long time, so we probably want to 
    # save the matrix file in the data that are loaded to start with
    flt.run()
else:
    print 'using existing BBR file'



Which is better?

In [ ]:
def calc_MI(x, y, bins=10):
    # compute mutual information between two variables
    c_xy = numpy.histogram2d(x, y, bins)[0]
    mi = sklearn.metrics.mutual_info_score(None, None, contingency=c_xy)
    return mi

crfile=meanbold_unwarped_file.replace('.nii.gz','_reg2t1.nii.gz')
bbrfile=meanbold_unwarped_file.replace('.nii.gz','_bbreg2t1.nii.gz')
maskvox=numpy.where(maskdata)
bcfdata=nibabel.load(bfcfile).get_data()[maskvox]
crdata=nibabel.load(crfile).get_data()[maskvox]
bbrdata=nibabel.load(bbrfile).get_data()[maskvox]
if 1:
    print 'Mutual information with target:'
    mi_bbr=calc_MI(bcfdata,bbrdata)
    print 'BBR:',mi_bbr
    mi_cr=calc_MI(bcfdata,crdata)
    print 'corratio:',mi_cr

Now let's register our functional data to the MNI template, and generate a composite transform.  Start out using FLIRT.

In [ ]:
mnimat_flirt=bfcfile.replace('.nii.gz','_reg2mni.mat')
if not os.path.exists(mnimat_flirt):
    flt = fsl.FLIRT(cost_func='normcorr',dof=12)
    flt.inputs.in_file = bfcfile
    flt.inputs.reference = os.path.join(os.getenv('FSLDIR'),
                            'data/standard/MNI152_T1_2mm_brain.nii.gz')
    flt.inputs.out_file = bfcfile.replace('.nii.gz','_reg2mni.nii.gz')
    flt.inputs.out_matrix_file = mnimat_flirt
    flt.run()
else:
    print 'using existing FLIRT MNI registration'

In [ ]:
if 0: #not os.path.exists(bfcfile.replace('.nii.gz','_nlreg2mni_warped.nii.gz')):

    flt = fsl.FNIRT()
    flt.inputs.in_file = bfcfile
    flt.inputs.affine_file=mnimat_flirt
    flt.inputs.ref_file = os.path.join(os.getenv('FSLDIR'),
                            'data/standard/MNI152_T1_2mm_brain.nii.gz')
    flt.inputs.warped_file = bfcfile.replace('.nii.gz','_nlreg2mni_warped.nii.gz')
    flt.inputs.field_file = bfcfile.replace('.nii.gz','_nlreg2mni_field')
    flt.run()
else:
    print 'using existing FLIRT MNI registration'

Use ANTS to register to MNI152 T1

In [ ]:
from nipype.interfaces import ants
reg = ants.Registration()
reg.inputs.fixed_image = os.path.join(os.getenv('FSLDIR'),
                            'data/standard/MNI152_T1_2mm_brain.nii.gz')
reg.inputs.moving_image = bfcfile
reg.inputs.output_warped_image = bfcfile.replace('.nii.gz','_ANTSreg2mni_warped.nii.gz')
reg.inputs.transforms = ['Translation', 'Rigid', 'Affine', 'SyN']
reg.inputs.transform_parameters = [(0.1,), (0.1,), (0.1,), (0.2, 3.0, 0.0)]
reg.inputs.number_of_iterations = ([[10000, 111110, 11110]]*3 +
                                    [[100, 50, 30]])
reg.inputs.dimension = 3
reg.inputs.write_composite_transform = True
#reg.inputs.collapse_output_transforms = False
reg.inputs.metric = ['Mattes'] * 3 + [['Mattes', 'CC']]
reg.inputs.metric_weight = [1] * 3 + [[0.5, 0.5]]
reg.inputs.radius_or_number_of_bins = [32] * 3 + [[32, 4]]
reg.inputs.sampling_strategy = ['Regular'] * 3 + [[None, None]]
reg.inputs.sampling_percentage = [0.3] * 3 + [[None, None]]
reg.inputs.convergence_threshold = [1.e-8] * 3 + [-0.01]
reg.inputs.convergence_window_size = [20] * 3 + [5]
reg.inputs.smoothing_sigmas = [[4, 2, 1]] * 3 + [[1, 0.5, 0]]
reg.inputs.sigma_units = ['vox'] * 4
reg.inputs.shrink_factors = [[6, 4, 2]] + [[3, 2, 1]]*2 + [[4, 2, 1]]
reg.inputs.use_estimate_learning_rate_once = [True] * 4
reg.inputs.use_histogram_matching = [False] * 3 + [True]
reg.inputs.initial_moving_transform_com = True
reg.inputs.terminal_output='none' #ANTs spits out a LOT of info 
reg.inputs.output_transform_prefix = os.path.join(anatomydir,"ANTS_")
antsresult=reg.run()

In [ ]:
mnimaskdata=nibabel.load(os.path.join(os.getenv('FSLDIR'),
            'data/standard/MNI152_T1_2mm_brain_mask.nii.gz')).get_data()
mnimaskvox=numpy.where(mnimaskdata)
mnidata=nibabel.load(os.path.join(os.getenv('FSLDIR'),
            'data/standard/MNI152_T1_2mm_brain.nii.gz')).get_data()[mnimaskvox]

flirtimg=nibabel.load(bfcfile.replace('.nii.gz',
            '_reg2mni.nii.gz'))
flirtdata=flirtimg.get_data()[mnimaskvox]
fnirtimg=nibabel.load(bfcfile.replace('.nii.gz',
            '_nlreg2mni_warped.nii.gz'))
fnirtdata=fnirtimg.get_data()[mnimaskvox]
print 'Mutual information with target:'
mi_flirt=calc_MI(mnidata,flirtdata)
print 'FLIRT:',mi_flirt
mi_fnirt=calc_MI(mnidata,fnirtdata)
print 'FNIRT:',mi_fnirt

Plot outlines onto template

In [ ]:
mniimg=nibabel.load(os.path.join(os.getenv('FSLDIR'),
                            'data/standard/MNI152_T1_2mm_brain.nii.gz'))

mask_display=nilearn.plotting.plot_epi(mniimg,cmap='gray')
mask_display.add_edges(flirtimg) 

In [ ]:
mask_display=nilearn.plotting.plot_epi(mniimg,cmap='gray')
mask_display.add_edges(fnirtimg) 

Apply FNIRT warp to EPI data

In [ ]:
unwarped_reg2mni=bold_file.replace('.nii.gz','_nlreg2mni.nii.gz')
if 0: #not os.path.exists(unwarped_reg2mni):
    aw = fsl.ApplyWarp()
    aw.inputs.in_file = bold_file
    aw.inputs.ref_file = os.path.join(os.getenv('FSLDIR'),
                                'data/standard/MNI152_T1_2mm_brain.nii.gz')
    aw.inputs.field_file = bfcfile.replace('.nii.gz','_nlreg2mni_field.nii.gz')
    aw.inputs.postmat=mnimat_flirt
    aw.inputs.out_file=unwarped_reg2mni
    aw.run()


Warp the gray matter mask back into the functional space, so we can use it for the analysis

In [ ]:
# first invert the warp from the bbrmat
invt = fsl.ConvertXFM()
invt.inputs.in_file = bbrmat
invt.inputs.invert_xfm = True
invt.inputs.out_file = bbrmat.replace('.mat','_inv.mat')
if not os.path.exists(invt.inputs.out_file):
    invt.run()
    
applyxfm=fsl.ApplyXfm()
applyxfm.inputs.in_file = os.path.join(anatomydir,'sub00001_ses018_T1w_001_brain_FAST_pve_1.nii.gz')
applyxfm.inputs.in_matrix_file = invt.inputs.out_file
applyxfm.inputs.out_file = os.path.join(bolddir,'gray_matter_mask.nii.gz')
applyxfm.inputs.reference = meanbold_unwarped_file
applyxfm.inputs.apply_xfm = True
if not os.path.exists(applyxfm.inputs.out_file):
    result = applyxfm.run()

gmthresh=0.25
if not os.path.exists(os.path.join(bolddir,'gray_matter_mask_bin.nii.gz')):
    gm=nibabel.load(os.path.join(bolddir,'gray_matter_mask.nii.gz'))
    gmdata=gm.get_data()
    gmdata[gmdata<gmthresh]=0
    gmdata[gmdata>0]=1
    gmbin=nibabel.Nifti1Image(gmdata,gm.get_affine())
    gmbin.to_filename(os.path.join(bolddir,'gray_matter_mask_bin.nii.gz'))